# Robotic Manipulator example
In this notebook we will walk through how the model of a robotic manipulator can be found as well as how the robotix mainpulator environment works

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

## Findig the equations of motion
We will use the symbolic python library in order to find the equations of motion for the manipulator

In [2]:
import numpy as np
from sympy import *
from sympy.matrices import *
from sympy.physics.mechanics import *
init_session()

IPython console for SymPy 1.0 (Python 3.6.0-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at http://docs.sympy.org/1.0/


### Function for calculating DH translation matrices
The [deviant hartenberg](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters) parameters are the four parameters associated with a particular convention for attaching reference frames to the links of a spatial kinematic chain, or robot manipulator. The convention can be represented as four transformation matrices. 
$$
\operatorname{Rot}_{z_{n - 1}}(\theta_n)
  =
\left[
\begin{array}{ccc|c}
    \cos\theta_n & -\sin\theta_n & 0 & 0 \\
    \sin\theta_n &  \cos\theta_n & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    \hline
    0 & 0 & 0 & 1
  \end{array}
\right]
, \quad
\operatorname{Trans}_{z_{n - 1}}(d_n)
  =
\left[
\begin{array}{ccc|c}
    1 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 1 & d_n \\
    \hline
    0 & 0 & 0 & 1
  \end{array}
\right]
$$
$$
\operatorname{Trans}_{x_n}(r_n)
  =
\left[
\begin{array}{ccc|c}
    1 & 0 & 0 & r_n \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    \hline
    0 & 0 & 0 & 1
  \end{array}
\right]
, \quad
\operatorname{Rot}_{x_n}(\alpha_n)
  =
\left[
\begin{array}{ccc|c}
    1 & 0 & 0 & 0 \\
    0 & \cos\alpha_n & -\sin\alpha_n & 0 \\
    0 & \sin\alpha_n & \cos\alpha_n & 0 \\
    \hline
    0 & 0 & 0 & 1
  \end{array}
\right]
$$
Combinig these matrices we get the final transformation matrix for each link given as:
$$
\operatorname{}T_n = 
\operatorname{Rot}_{z_{n - 1}}(\theta_n)
\operatorname{Trans}_{z_{n - 1}}(d_n)
\operatorname{Trans}_{x_n}(r_n)
\operatorname{Rot}_{x_n}(\alpha_n)
  =
\left[
\begin{array}{ccc|c}
    \cos\theta_n & -\sin\theta_n \cos\alpha_n & \sin\theta_n \sin\alpha_n & r_n \cos\theta_n \\
    \sin\theta_n & \cos\theta_n \cos\alpha_n & -\cos\theta_n \sin\alpha_n & r_n \sin\theta_n \\
    0 & \sin\alpha_n & \cos\alpha_n & d_n \\
    \hline
    0 & 0 & 0 & 1
  \end{array}
\right]
$$

In [3]:
# Calculates transformation matrix
def DH(theta, d, a, alpha):
    A1 = Matrix([[cos(theta), -sin(theta), 0, 0], [sin(theta), cos(theta), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    A2 = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, d], [0, 0, 0, 1]])
    A3 = Matrix([[1, 0, 0, a], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    A4 = Matrix([[1, 0, 0, 0], [0, cos(alpha), -sin(alpha), 0], [0, sin(alpha), cos(alpha), 0], [0, 0, 0, 1]])
    return A1*A2*A3*A4

# Calculate multiple transformation matrixes
def DH_all(parameters):
    T = []
    for i , pi in enumerate(parameters):
        T.append(DH(pi[0], pi[1], pi[2], pi[3]))
    return T

### Calculate mass center distance from inertial cordinate frame
Given a transformation matrix from a referance frame $0$ to referance frame $1$ denoted $T^0_1$ and a vector in the referanve frame $1$ given as $r^1 = [x, y, z]^T$. We can then compute the vector in refrence frame $0$ as:
$$
    r^0 = T^0_1 r^1
$$
The following function performs these calculations when given the center of mass in the local referancce frame, it computes the center of mass in the inertial referance frame

In [6]:
'''
 - T are the transformation matrices
 - r_l are the vectors of the center of mass in the loal referance frame
 - r_i are the vectors of the center of mass in the inertial frame
'''
def center_of_mass(T, r_l):
    T0i = eye(4)
    r_i = []
    for i , li in enumerate(r_l):
        T0i *= T[i]
        ri.append(T0i*li)
    return r_i

### Function for calculating jacobian
The time derivative of the kinematics equations yields the Jacobian of the robot, which relates the joint rates to the linear and angular velocity of the end-effector. The principle of virtual work shows that the Jacobian also provides a relationship between joint torques and the resultant force and torque applied by the end-effector. Singular configurations of the robot are identified by studying its Jacobian. 

The robotic linear velocity jacobian for the DH convention is defined as $J_v = [J_{v_1}, \dots, J_{v_i}]$ where:

$$
J_{v_i} = 
\begin{cases}
  z_{i - 1} \times (o_n - o_{i-1}) \quad & \text{for revolute joint}\\
  Z_{i-1} \quad & \text{for prismatic joints}
\end{cases}
$$

The robotic angular velocity jacobian for the DH convention is defined as $J_\omega = [J_{\omega_1}, \dots, J_{\omega_i}]$ where:

$$
J_{v_i} = 
\begin{cases}
  z_{i - 1} \quad & \text{for revolute joint}\\
  0 \quad & \text{for prismatic joints}
\end{cases}
$$

In [10]:
'''
 The inputs to the function are:
 - T are the transformation matrices
 - joints are prismatic p or revolute r 
 - r_cm is the center of mass of each joint in the inertial frame
 
 The function returns one list containg the linear velocity jacobians for the to the centers of mass, 
 and one list containig the angular velocity jacobians to the centers of mass. 
'''

import copy
def jacobian(T, joints, r_cm):
    Tn = eye(4)
    for i , Ti in enumerate(T):
        Tn *= Ti
    Jvi = []
    Jwi = []
    for i in range(len(r_cm)):
        Jv = zeros(3, len(r_cm))
        Jw = zeros(3, len(r_cm))
        temp = eye(4)
        rci = r_cm[i]
        for j in range(i + 1):
            zj = temp[0:3, 2]
            if joints[j] == 'p': #Prismatic
                Jv[0:3, j] = zj
                Jw[0:3, j] = zeros(3, 1)
            if joints[j] == 'r': #Revolute
                Jv[0:3, j] = transpose(Matrix(np.cross(transpose(zj), transpose(rci[0:3, 0] - temp[0:3, 3]))))
                Jw[0:3, j] = zj
            temp *= T[j]
        Jvi.append(simplify(copy.deepcopy(Jv)))
        Jwi.append(simplify(copy.deepcopy(Jw)))
    return Jvi, Jwi

### Function for calculating equations of motion

In [8]:
def christoffel(D, i, j, k, q):
    temp = diff(D[k, j], q[i]) + diff(D[k, i], q[j]) - diff(D[i, j], q[k]) 
    return simplify(temp/2)

def EoM(Jvi, Jwi, I, T, rc, m, q, g):
    D = zeros(len(q), len(q))
    Ri = eye(3)
    for i , Ti in enumerate(T):
        mi = m[i]
        Ri *= Ti[0:3, 0:3]
        Ii = I[i]
        D += mi*transpose(Jvi[i])*Jvi[i] + transpose(Jwi[i])*Ri*Ii*transpose(Ri)*Jwi[i]
    D = simplify(D)
    C = zeros(len(q), len(q))
    for i in range(len(q)):
        for j in range(len(q)):
            for k in range(len(q)):
                C[k, j] += christoffel(D, i, j, k, q)*diff(q[i], Symbol('t'))
    C = simplify(C)
    G = zeros(len(q), 1)
    P = zeros(1,1)
    for i , rci in enumerate(rc):
        P += m[i]*transpose(g)*rci[0:3, 0]
    for i, qi in enumerate(q):
        G[i, 0] = diff(P, qi)
    K = 0
    for i in range(len(q)):
        for j in range(len(q)):
            K += D[i, j]*diff(q[i], Symbol('t'))*diff(q[j], Symbol('t'))
    return D, C, G, simplify(K/2), simplify(P[0,0])

# Finding equations of motion for 3 link manipulator

In [11]:
#Find transformation matrices
q1, q2, q3 = dynamicsymbols('q1 q2 q3')
d3, a3 = 0, 1

T = DH_all([[q1, 0, 0, pi/2],[0, q2, 0, -pi/2],[q3, d3, a3, 0]])

#Define center of mass of each link
m1, m2, m3 = 5, 5, 5
lmx1, lmy1, lmz1, lmx2, lmy2, lmz2, lmx3, lmy3, lmz3 = 0, 0.5, 0, 0, 0, 0.5, 0, 0, 0.5

lm1 = Matrix([lmx1, lmy1, lmz1, 1])
lm2 = Matrix([lmx2, lmy2, lmz2, 1])
lm3 = Matrix([lmx3, lmy3, lmz3, 1])

#Find center of mass in inertia frame
lm01 = simplify(T[0]*lm1)
lm02 = simplify(T[0]*T[1]*lm2)
lm03 = simplify(T[0]*T[1]*T[2]*lm3)

#Define inerti tensor of each link
Ix1, Iy1, Iz1, Ix2, Iy2, Iz2, Ix3, Iy3, Iz3 = 0, 1/12*m1*0.5**2, 0, 0, 0, 1/12*m1*0.5**2, 0, 0, 1/12*m1*0.5**2
I1 = Matrix([[Ix1, 0, 0], [0, Iy1, 0], [0, 0, Iz1]])
I2 = Matrix([[Ix2, 0, 0], [0, Iy2, 0], [0, 0, Iz2]])
I3 = Matrix([[Ix3, 0, 0], [0, Iy3, 0], [0, 0, Iz3]])

#find Jacobian
Jvi, Jwi = jacobian(T, ['r','p','r'], [lm01, lm02, lm03])

#find equatgions of motion
g = Matrix([[0],[9.81], [0]])
D, C, G, K, P = EoM(Jvi, Jwi, [I1, I2, I3], T, [lm01, lm02, lm03], [m1, m2, m3], [q1, q2, q3], g)

In [13]:
q1, q2, q3 = dynamicsymbols('q1 q2 q3')
L = K - P
LM = LagrangesMethod(L, [q1, q2, q3])
eom = LM.form_lagranges_equations()

LM.mass_matrix_full

⎡1  0  0                       0                              0               
⎢                                                                             
⎢0  1  0                       0                              0               
⎢                                                                             
⎢0  0  1                       0                              0               
⎢                                                                             
⎢                2                                                            
⎢0  0  0  10.0⋅q₂ (t) - 10.0⋅q₂(t)⋅sin(q₃(t)) + 5.3125  -5⋅cos(q₃(t))  -5.0⋅q₂
⎢                                                                             
⎢0  0  0                 -5⋅cos(q₃(t))                       10               
⎢                                                                             
⎣0  0  0    -5.0⋅q₂(t)⋅sin(q₃(t)) + 5.10416666666667    -5⋅cos(q₃(t))         

            0                    ⎤
                

# Find equations of motion for 2 link maipulator

In [14]:
M = LM.mass_matrix_full
F = LM.forcing_full

q1d, q2d, q3d = dynamicsymbols('q1d q2d q3d')
F = F.subs(diff(q1, Symbol('t')), q1d)
F = F.subs(diff(q2, Symbol('t')), q2d)
F = F.subs(diff(q3, Symbol('t')), q3d)

In [15]:
from numpy.linalg import solve
state_vector = [q1, q2, q3, q1d, q2d, q3d]

C = C.subs(diff(q1, Symbol('t')), q1d)
C = C.subs(diff(q2, Symbol('t')), q2d)
C = C.subs(diff(q3, Symbol('t')), q3d)

M_func = lambdify(state_vector, M)
F_func = lambdify(state_vector, F)

D_func = lambdify(state_vector, D)
C_func = lambdify(state_vector, C)
G_func = lambdify(state_vector, G)


Kp = np.diag([100, 100, 100])
Kd = np.diag([100, 100, 100])
x_d = [3.14/2, 1, 0, 0, 0, 0]

def dx(x, t, args):
    
    #PD-regulator 
    q_tilde = np.subtract(np.matrix(x), np.matrix(x_d)).T
    u = -Kp*q_tilde[0:3, 0] - Kd*q_tilde[3:6, 0] + G_func(*x)
    
    #Joint space inverse dynamics
    a, b, c = 0.5, 2, 0.25
    qd = [np.sin(a*t), np.sin(b*t), np.sin(c*t), a*np.cos(a*t), b*np.cos(b*t), c*np.cos(c*t)]
    q_tilde = np.subtract(np.matrix(x), np.matrix(qd)).T
    q_dd_d = np.matrix([[-a*a*np.sin(a*t)],[-b*b*np.sin(b*t)],[-c*c*np.sin(c*t)]])
    
    aq = q_dd_d - Kp*q_tilde[0:3, 0] - Kd*q_tilde[3:6, 0]
    u = D_func(*x)*aq + C_func(*x)*np.matrix(x[3:6]).T + G_func(*x)
    
    tau = numpy.concatenate((np.matrix([[0],[0],[0]]),u))
    #find state derivatives
    return (solve(M_func(*x), F_func(*x) + np.array(tau))).T[0]

In [16]:
from scipy.integrate import odeint
parameter_vals = 0
t = np.linspace(0, 15, 1000)                        # Time vector
x0 = [0, 0, 0, 0, 0, 0]
y = odeint(dx, x0, t, args=(parameter_vals,))       # Actual integration

In [17]:
import matplotlib.pyplot as plt
pq1 = plt.plot(t, y[:, 0:3])
plt.show()

In [18]:
#from numpy import zeros, cos, sin, arange, around
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
plt.rcParams['animation.ffmpeg_path'] = 'C:/Program Files (x86)/ffmpeg/bin/ffmpeg.exe'

P01 = lambdify([q1, q2, q3], (T[0])[0:3, 3])
P02 = lambdify([q1, q2, q3], (T[0]*T[1])[0:3, 3])
P03 = lambdify([q1, q2, q3], (T[0]*T[1]*T[2])[0:3, 3])


pos1 = zeros(3, len(t))
pos2 = zeros(3, len(t))
pos3 = zeros(3, len(t))
for i in range(len(t)):
    pos1[:, i] = P01(*y[i, 0:3])
    pos2[:, i] = P02(*y[i, 0:3])
    pos3[:, i] = P03(*y[i, 0:3])

filename = None
#filename = 'open-loop.mp4'


# First set up the figure, the axis, and the plot element we want to animate
plt.close()
fig = plt.figure()
ax = plt.axes(xlim=(-2, 2), ylim=(-2, 2), aspect='equal')
time_text = ax.text(0.04, 0.9, '', transform=ax.transAxes)
line, = ax.plot([], [], lw=2, marker='o')

# initialization function: plot the background of each frame
def init():
    time_text.set_text('')
    line.set_data([], [])
    return time_text, line,

# animation function.  This is called sequentially
def animate(i):
    time_text.set_text('time = {:2.2f}'.format(t[i]))
    x = [0, 0, 0]
    y = [0, 0, 0]
    x[0] = pos1[0, i]
    y[0] = pos1[1, i]
    x[1] = pos2[0, i]
    y[1] = pos2[1, i]
    x[2] = pos3[0, i]
    y[2] = pos3[1, i]
    
    line.set_data((x, y))
    return time_text, line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=len(t), init_func=init,
                                   interval=t[-1] / len(t) * 1000, blit=True, repeat=False)
    
if filename is not None:
    anim.save(filename, fps=30, extra_args=['-vcodec', 'libx264'])

plt.show()